## How are in demand skills trending for Data Analysts?

### Methodology

1. Aggregate skill counts monthly
2. Re-analyze based on percentage of job totals 
3. Plot the monthly skill demand

In [19]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast


from datasets import load_dataset

# Load the dataset from Hugging Face
dataset = load_dataset("lukebarousse/data_jobs", split="train")

# Convert to a pandas DataFrame
df = dataset.to_pandas()

#data clean up
df['job_posted_date'] = pd.to_datetime(df.job_posted_date )
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else (x))

# Display the first few rows
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"[r, python, sql, nosql, power bi, tableau]","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"[python, sql, c#, azure, airflow, dax, docker,...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"[python, c++, java, matlab, aws, tensorflow, k...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"[bash, python, oracle, aws, ansible, puppet, j...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


In [21]:
#filter for Nigeria data analyst roles

df_da_ng = df[(df['job_title'] == 'Data Analyst') & (df['job_country']== 'Nigeria')].copy()

In [22]:
#Aggregate by month
df_da_ng['job_posted_month_no'] = df_da_ng['job_posted_date'].dt.month

In [23]:
df_da_ng_explode = df_da_ng.explode('job_skills')

In [25]:
# get a pivot table
df_da_ng_pivot = df_da_ng_explode.pivot_table(index = 'job_posted_month_no', columns = 'job_skills', aggfunc = 'size', fill_value=0)

# create a new row of total and aggregate by sum
df_da_ng_pivot.loc['Total'] = df_da_ng_pivot.sum()

# sort data
df_da_ng_pivot[df_da_ng_pivot.loc['Total'].sort_values(ascending = False).index]

# dropped the total row so i can plot
df_da_ng_pivot = df_da_ng_pivot.drop('Total')

df_da_ng_pivot

job_skills,angular,bigquery,c++,elasticsearch,excel,firebase,github,go,hadoop,java,...,sap,sas,scala,sheets,spreadsheet,spss,sql,tableau,visual basic,word
job_posted_month_no,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,1,0,1,2,0,0
2,0,0,0,0,2,0,0,1,0,0,...,0,2,0,0,0,0,2,2,0,1
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,1,1,0,2,0
4,0,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,1,3,2,0,2
5,0,0,1,0,4,0,1,0,1,1,...,0,2,1,2,0,1,4,3,0,0
6,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,2,2,0,0
7,0,0,0,0,3,2,0,0,0,0,...,0,0,0,0,0,0,5,3,0,0
8,0,1,0,1,2,0,0,1,0,0,...,0,0,0,0,0,0,3,3,0,0
9,0,0,0,0,6,0,0,0,0,0,...,1,0,0,0,1,0,4,2,0,2


want to get df_da_ng_pivot from a count to a percentage

In [26]:
da_total = df_da_ng.groupby('job_posted_month_no').size()

In [28]:
df_da_ng_percent = df_da_ng_pivot.div(da_total/100, axis = 0)
df_da_ng_percent

job_skills,angular,bigquery,c++,elasticsearch,excel,firebase,github,go,hadoop,java,...,sap,sas,scala,sheets,spreadsheet,spss,sql,tableau,visual basic,word
job_posted_month_no,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,0.000000,0.0,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,20.000000,0.000000,20.000000,40.000000,0.0,0.000000
2,0.000000,0.0,0.000000,0.0,33.333333,0.000000,0.000000,16.666667,0.000000,0.000000,...,0.000000,33.333333,0.000000,0.000000,0.000000,0.000000,33.333333,33.333333,0.0,16.666667
3,0.000000,0.0,0.000000,0.0,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,20.000000,0.000000,40.0,0.000000
4,0.000000,0.0,0.000000,0.0,62.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,12.500000,37.500000,25.000000,0.0,25.000000
5,0.000000,0.0,14.285714,0.0,57.142857,0.000000,14.285714,0.000000,14.285714,14.285714,...,0.000000,28.571429,14.285714,28.571429,0.000000,14.285714,57.142857,42.857143,0.0,0.000000
6,0.000000,0.0,0.000000,0.0,75.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,50.000000,0.0,0.000000
7,0.000000,0.0,0.000000,0.0,33.333333,22.222222,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,55.555556,33.333333,0.0,0.000000
8,0.000000,20.0,0.000000,20.0,40.000000,0.000000,0.000000,20.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000,60.000000,0.0,0.000000
9,0.000000,0.0,0.000000,0.0,54.545455,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.090909,0.000000,0.000000,0.000000,9.090909,0.000000,36.363636,18.181818,0.0,18.181818
